# Patterns for Cleaner Python

## Assertion

assertion is about unrecoverable errors; while exception can happen and needs to be handled. assertion helps debug but not runtime errors. the python interpreter translate 

```python
assert expression1, expression2
```

to

```python
if __debug__:       # global variable, False when optimized
    if not expression1:
        raise AssertionError(expression2)
```


assert is globally disabled with -o, -oo command line swithes, and PYTHONOPTIMIZE enviroment variable in CPython, in which case assert is compiled away. So be sure not use assert for prod validation of data.

In [4]:
# never pass a tuple as first argument in assert, it will never fail:
assert (1==2, 'this will never fail')

## Complacent Comma Placement

one item per line, use comma.

In [5]:
names = [
    'alice',
    'bob',
    'charlie',
    'jane',
]

## Context Managers and the `with` Statement

`with` helps with resource management. It simplifies explicity 'try... finally' statement.

### supporting `with` in objects

implement `__enter__` and `__exit__` context manager. python calls `__enter__` when execution enters the context of `with`, then call `__exit__` when resource got freed.



In [6]:
class ManageFile:
    def __init__(self, name):
        self.name = name

    def __enter__(self):
        self.file = open(self.name, 'w')
        return self.file

    def __exit__(self):
        if self.file:
            self.file.close()

another way is to use std library `contextlib.contextmanager` decorator to define a generator-based factory function for resource. It will automatically support with statement.

In [8]:
from contextlib import contextmanager

@contextmanager
def managed_file(name):
    try:
        f = open(name, 'w')
        yield f
    finally:
        f.close()

with managed_file('hello.txt') as f:
    f.write('hw')

In [11]:
class Indenter:
    def __init__(self):
        self.level = 0

    def __enter__(self):
        self.level += 1
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.level -= 1

    def print(self, text):
        print('    ' * self.level + text)

with Indenter() as indent:
    print(indent)
    indent.print('hi!')
    with indent:
        indent.print('hello')
        with indent:
            indent.print('world')
    indent.print('hey') 

None


AttributeError: 'NoneType' object has no attribute 'print'

In [10]:
indent = Indenter()
indent.print('1')

1


## Underscores

### single leading `_`

suggested private 

### double leading `__`

python would change such variable name to a new one. a form of stronger suggested private. check examples

### double leading/trailing `__`

python would not mangle such variables, leave it for magic methods

### single `_`

just a variable name, REPL use it to reference last variable


## String formating

1. old c-style 
2. python style
3. formatted string literal
4. template string (good for user input, for security reasons)

In [19]:
errno = 50159747054
name = 'Bob'

# c-style
print('hello, %s' % name)
print('%x' % errno)
print('hey %s, there is 0x%x error!' % (name, errno))
print('hey %(name)s, there is a 0x%(errno)x error!' % {'name':name, 'errno': errno})

# python-style
print('hello, {}'.format(name))
print('hey {name}, there is a 0x{errno:x} error!'.format(name=name, errno=errno))

# formatted string literals
print(f'Hello, {name}')
a, b = 5, 10
print(f'can do calculations: {a + b} and {2 * (a+b)}')
print(f'hey {name}, there is a {errno:#x} error!')

# template string
from string import Template
t = Template('hey, $name')
t.substitute(name=name)

templ_string = 'hey $name, there is a $error error'
Template(templ_string).substitute(name=name, error=hex(errno))

# template string is safe
PASSWORD = 'pass'
class Error:
    def __init__(self):
        pass
err = Error()
user_input = '{error.__init__.__globals__[PASSWORD]}'
user_input.format(error=err)

user_input = '${error.__init__.__globals__[PASSWORD]}'
try:
    Template(user_input).substitute(error=err)
except ValueError as e:
    print(e)

hello, Bob
badc0ffee
hey Bob, there is 0xbadc0ffee error!
hey Bob, there is a 0xbadc0ffee error!
hello, Bob
hey Bob, there is a 0xbadc0ffee error!
Hello, Bob
can do calculations: 15 and 30
hey Bob, there is a 0xbadc0ffee error!
Invalid placeholder in string: line 1, col 1


In [21]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


# Effective Functions

## python function are first class



In [23]:
def yell(text):
    return text.upper() + '!'

bark = yell

# functions passed to other functions
def greet(func):
    greeting = func('Hi, I am a Python program')
    print(greeting)

greet(bark)

# function can be nested
def speak(text):
    def whisper(t):
        return t.lower() + '...'
    return whisper(text)

# but whisper does not exist outside speak
try:
    whisper('Yo')
except NameError as e:
    print(e)

try:
    speak.whisper
except AttributeError as e:
    print(e) 

HI, I AM A PYTHON PROGRAM!
name 'whisper' is not defined
'function' object has no attribute 'whisper'


### functions can capture local state

the inner functions can capture parent function's state. Functions that do this are called lexical closures (or just closures, for short). A closure remembers the values from its enclosing lexical scope even when the program flow is no longer in that scope.

you can pre-config some behaviors to the function you return.

In [25]:
def get_speak_func(text, volume):
    def whisper():
        return text.lower() + '...'
    def yell():
        return text.upper() + '...'
    if volume > 0.5:
        return yell
    else:
        return whisper

print(get_speak_func('Hello', 0.7)())

'HELLO...'

### objects can behave like functions through `__call__`

In [27]:
class Adder:
    def __init__(self, n):
        self.n = n
    def __call__(self, x):
        return self.n + x

plus_3 = Adder(3)
print(plus_3(4))
print(callable(plus_3))

7
True


## Lambdas

it is a single-expression function that are not necessarily bound to a name.

it has lexical scope that captures context


In [3]:
(lambda x, y: x + y)(3, 4)

def make_adder(n):
    return lambda x: x + n

plus_3 = make_adder(3)
plus_5 = make_adder(5)

plus_3(4)

7

## Decorators

* It uses the * and ** operators in the wrapper closure definition to collect all positional and keyword arguments and stores them in variables (args and kwargs).

* The wrapper closure then forwards the collected arguments to the original input function using the * and ** “argument unpacking” operators.

* use functional.wrap to keep meta data



In [4]:
# multiple decorator order

def strong(func):
    def wrapper():
        return 'strong' + func()
    return wrapper

def emphasis(func):
    def wrapper():
        return 'emphy' + func()
    return wrapper

@strong
@emphasis
def greet():
    return "Hello"

greet()

'strongemphyHello'

In [ ]:
# pass arguments

def proxy(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

In [5]:
# loss of meta data
def greet():
    """ greetings """

def uppercase(func):
    def wrapper():
        return func().upper()
    return wrapper

decor_greet = uppercase(greet)
decor_greet.__name__

'wrapper'

In [6]:
import functools

def uppercase(func):
    @functools.wraps(func)
    def wrapper():
        return func().upper()
    return wrapper

@uppercase
def greet():
    """ greetter"""
    return 'hello!'

greet.__name__

'greet'

## `*args` and `**kwargs`

forwarding optional and keyword args.

In [ ]:
def foo(x, *args, **kwargs):
    kwargs['name'] = 'Alice'
    new_args = args + ('extra', )
    bar(x, *new_args, **kwargs)

## Function Argument Unpacking

all iterable, including generaotr expressions can be unpacked with *. all keyword arguments dictionary can be unpacked with **. Note * on kwargs will unpack keys.

In [11]:
def print_3num(x, y, z):
    print ('<%s, %s, %s>' % (x, y, z))

tup_vec = (1, 0, 1)
print_3num(*tup_vec)

gen_vec = (x * x for x in range(3))
print_3num(*gen_vec)

dic_vec = {'x': 1, 'y':3, 'z': 5}
print_3num(**dic_vec) # unpact values
print_3num(*dic_vec) # unpact keys

<1, 0, 1>
<0, 1, 4>
<1, 3, 5>
<x, y, z>


# Classes and OOP
## is vs ==
* `==` checks equality
* `is` checks address

In [2]:
a = [1, 2, 3]
b = a

print(a == b)
print(a is b)

c = list(a)
print(c == a)
print (c is a)

True
True
True
False


## every class needs a `__repr__`

`__str__` is not triggered when stack unwinding. print will call `__repr__` if __str__ is not implemented. 

In [6]:
class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage

    def __repr__(self):
        return '__repr__ for Car'

    def __str__(self):
        return '__str__ for Car'

my_car = Car('red', 1234)
print(my_car)
print('{}'.format(my_car))
print(str([my_car]))
my_car

__str__ for Car
__str__ for Car
[__repr__ for Car]


__repr__ for Car

In [9]:
# the !r conversion flag make sure the output string uses repr(self.color) instead of str(self.color)
def __repr__(self):
    return (f'{self.__class__.__name__}('f'{self.color!r}, {self.mileage!r})')

## define own exception

In [10]:
class NameTooshort(ValueError):
    pass

def validate(name):
    if len(name) < 10:
        raise NameTooshort(name)

validate('a')

NameTooshort: a

## cloning objects

build in copy is shallow
```python
newlist = list(originallist)
newdict = dict(originaldict)
newset = set(originalset)
```

use `deepcopy` to deepcopy, use `copy` to explicit shallow copy for customized types, use list/dict/set to shallow copy.

you can control the class behavior using `__copy__()` and `__deepcopy__()`

In [13]:
xs = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
ys = list(xs)
xs[1][0] = 'x'
print(ys)

import copy
zs = copy.deepcopy(xs)
xs[1][1]='u'
print(zs)

[[1, 2, 3], ['x', 5, 6], [7, 8, 9]]
[[1, 2, 3], ['x', 5, 6], [7, 8, 9]]


In [17]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return f'Point({self.x!r}, {self.y!r})'

a = Point(23, 42)
b = copy.copy(a)         # the primitive type shallow = deep

print(a is b)
a.x = 32

print(b)

False
Point(23, 42)


In [21]:
class Rectangle:
    def __init__(self, topleft, bottomright):
        self.topleft = topleft
        self.bottomright = bottomright

    def __repr__(self):
        return (f'Rectangle({self.topleft!r}, 'f'{self.bottomright!r})')

r1 = Rectangle(Point(0, 1), Point(5, 6))
r2 = copy.copy(r1)
r3 = copy.deepcopy(r1)

r1.topleft.x = 999
print(r1)
print(r2)
print(r3)


Rectangle(Point(999, 1), Point(5, 6))
Rectangle(Point(999, 1), Point(5, 6))
Rectangle(Point(0, 1), Point(5, 6))


## Abstract Base Classes

we need the to define the interface
1. instantiating the base class is impossible
2. forgetting implementing any interface methods will raise error

In [22]:
from abc import ABCMeta, abstractclassmethod

class Base(metaclass=ABCMeta):
    @abstractclassmethod
    def foo(self):
        pass

    @abstractclassmethod
    def bar(self):
        pass

class Concrete(Base):
    def foo(self):
        pass
    # no bar()

c = Concrete()

TypeError: Can't instantiate abstract class Concrete with abstract methods bar

## Namedtuples for read only dictionary

tuple is good, but you can not name index it. 
* 1st string property is the variable name, good for debug __repr__
* space delimited string is a syntax sugar for \[color, mileage\]

In [28]:
from collections import namedtuple

Car = namedtuple('Car', 'color mileage')
my_car = Car('red', 3000)
print(my_car.color)
print(my_car.mileage)
print(my_car[0])
tuple(my_car)

color, mileage = my_car
print(color)
print(mileage)
print(my_car)

red
3000
red
red
3000
Car(color='red', mileage=3000)


In [29]:
# property is const
my_car.color = 'blue'

AttributeError: can't set attribute

the proper way to subclass a named tuple and add more constant attributes are not subclass, but use `_fields`

In [31]:
ElectricCar = namedtuple('ElectricCar', Car._fields + ('charge',))
ElectricCar('red', 3222, 45.0)

ElectricCar(color='red', mileage=3222, charge=45.0)

there are many built-in helpers starts with `_`:
* `_asdict` turn into dictionary
* `_replace` shallow copy a tuple and allows to selectively replace some fields
* `_make` turns a iteratable (tuple, list, set) into a namedtuple

In [34]:
print(my_car._asdict())

print(my_car._replace(color='blue'))

print(Car._make(['red', 999]))

OrderedDict([('color', 'red'), ('mileage', 3000)])
Car(color='blue', mileage=3000)
Car(color='red', mileage=999)


## Class vs Instanve Variable Pitfalls

* class variables are shared among instances
* instance can read only class variables, but once it sets value, it will create a instance level varialbe, and shaddow the class variable 

In [40]:
class Dog:
    numlegs = 4
    def __init__(self, name):
        self.name = name

jack = Dog('jack')
jill = Dog('jill')

print(jack.numlegs, jill.numlegs, Dog.numlegs)

Dog.numlegs = 6
print(jack.numlegs, jill.numlegs, Dog.numlegs)

Dog.numlegs = 4
jack.numlegs = 6        # shaddows class property
print(jack.numlegs, jill.numlegs, Dog.numlegs)

print(jack.numlegs, jack.__class__.numlegs)


4 4 4
6 6 6
6 4 4
6 4


## Instance, Class and Static Methods 

* instance methods can access class itself through `self._class__` attribute, so that to modify class state
* class methods only has access to cls, it cannot modify state of instance, but it can modify class state across all instances.
* static method cannot modify object state or class state. it is restricted in what data they can access, they're primarily used as namespace your methods. 

In [43]:
class MyClass:
    def method(self):
        return 'instance method called', self

    @classmethod
    def classmethod(cls):
        return 'class method called', cls

    @staticmethod
    def staticmethod():
        return 'static method called'

obj = MyClass()
obj.method()

('instance method called', <__main__.MyClass at 0x7f303d3cab90>)

In [45]:
# alternative way to call obj.method(), explicity pass obj to self
MyClass.method(obj) 

('instance method called', <__main__.MyClass at 0x7f303d3cab90>)

In [47]:
# obj access self.__class__ attribute and call the class method
obj.classmethod()

('class method called', __main__.MyClass)

In [48]:
# obj access static method
obj.staticmethod()

'static method called'

In [50]:
MyClass.classmethod()

('class method called', __main__.MyClass)

In [52]:
MyClass.staticmethod()


'static method called'

In [53]:
MyClass.method()

TypeError: method() missing 1 required positional argument: 'self'

### class method for factory pattern

python allow one `__init__` per class, but class methods can make many constructors

In [9]:
import math

class Pizza:
    def __init__(self, radius, ingredients):
        self.radius = radius
        self.ingredients = ingredients
    def __repr__(self):
        return (f'Pizza({self.radius!r}, {self.ingredients!r})')

    @classmethod
    def margherita(cls):
        return cls(2, ['mozzarella', 'tomatoes'])
    @classmethod
    def prosciutto(cls):
        return cls(4, ['mozzarella', 'tomatoes', 'ham'])

    def area(self):
        return self.circle_area(self.radius)
    
    # not allow to modify class state
    @staticmethod
    def circle_area(r):
        return r ** 2 * math.pi

print(Pizza.margherita())

p = Pizza.prosciutto()
print(p)

print(p.area())
Pizza.circle_area(4)

Pizza(2, ['mozzarella', 'tomatoes'])
Pizza(4, ['mozzarella', 'tomatoes', 'ham'])
50.26548245743669


50.26548245743669

# Common Data Structure
## Dictionaries, Maps, and Hashtables

### dictionaries

python dictionaries keys can only be hashable (the object's hash value __hash__ does not change during the life, and obj can be compared by __eq__). tuple can be key only if the elements are immutable.

### `collections.OrderdDict` Remember the Insertion Order of keys

CPython 3.6+ dict will preserve the insertion order. Other version we need `OrderDict`.


In [3]:
import collections
d = collections.OrderedDict('one'=1, 'two'=2, 'three'=3)

SyntaxError: keyword can't be an expression (<ipython-input-3-f365dc43005b>, line 2)

### `collections.defaultdict` return default values for missing keys 

In [5]:
from collections import defaultdict
dd = defaultdict(list)

dd['dog'].append('rufus')
dd['dog'].append('kathrin')
dd['dog'].append('mr white')
dd['dog']

['rufus', 'kathrin', 'mr white']

### `collections.ChainMap` search multiple dictionies as a single mapping

lookups search the underlying dicts one by one until a key is found. Insertions, updates, deletions only affect the 1st mapping in the chain

In [6]:
from collections import ChainMap
d1 = {'one': 1, 'two': 2}
d2 = {'three': 3, 'four': 4}
chain = ChainMap(d1, d2)

chain

ChainMap({'one': 1, 'two': 2}, {'three': 3, 'four': 4})

### `types.MappingProxyType` read only dictionaries 

In [9]:
from types import MappingProxyType

writable = {'one': 1, 'two': 2}
readonly = MappingProxyType(writable)

writable['one'] = 42
readonly

mappingproxy({'one': 42, 'two': 2})

## Array Data Structures

### `list` mutable dynamic arrays


In [10]:
arr = [1, 2, 3]
del arr[1]

### `tuple` immutable containers

create copy to change content

In [12]:
arr = 1, 2, 3
print(arr)
arr + (23, )

(1, 2, 3)


(1, 2, 3, 23)

### `array.array` Basic Typed Arrays

memory effecient


In [13]:
import array

arr = array.array('f', (1.0, 1.5, 2.0, 2.5))
print(arr)

del arr[1]
arr.append(42.)

array('f', [1.0, 1.5, 2.0, 2.5])


### `str` Immutable Arrays of Unicode Characters

each character in a stirng is a str objevt of length 1 itself. string is Immutable so you cannot assign, del.

### `bytes` Immutable Arrays of Single bytes

bytes objects are immutable sequences of single bytes (integers in the range of 0-255). Conceptually they're similar to str objects, immutable arrays of bytes. there is a dedicated mutable byte array data type called bytearray that they can be unpacked into.


In [14]:
arr = bytes((0, 2, 3, 4))
print(arr[1])
print(arr)

## cannot do arr[1] = 23, del arr[1] bytes(300) <- out of 0, 255

2
b'\x00\x02\x03\x04'


### `bytearray` mutable arrays of single bytes

bytearry is like list to str, you can modify it freely.

In [16]:
arr = bytearray((0, 1, 2, 3))
arr[1] = 23
print(arr)
del arr[1]
print(arr)
arr.append(42)
print(array)

bytearray(b'\x00\x17\x02\x03')
bytearray(b'\x00\x02\x03')
bytearray(b'\x00\x02\x03*')


## Records, Structs, and Data Transfer Objects

### `dict` Simple mutable data Objects

### `tuple` immutable groups of Objects


In [19]:
import dis

print(dis.dis(compile("(23, 'a', 'b', 'c')", '', 'eval')))
dis.dis(compile("[23, 'a', 'b', 'c']", '', 'eval'))


1           0 LOAD_CONST               0 ((23, 'a', 'b', 'c'))
              2 RETURN_VALUE
None
  1           0 LOAD_CONST               0 (23)
              2 LOAD_CONST               1 ('a')
              4 LOAD_CONST               2 ('b')
              6 LOAD_CONST               3 ('c')
              8 BUILD_LIST               4
             10 RETURN_VALUE


### Custom class

the @property decorator can make the field read-inly.

### `collections.namedtuple`

it is immutable, cannot add new fields or modify once created.

In [20]:
from collections import namedtuple
from sys import getsizeof

p1 = namedtuple('point', 'x y z')(1, 2, 3)
p2 = (1, 2, 3)
print(getsizeof(p1))
print(getsizeof(p2))

80
80


### `typing.NamedTuple` Improved NamedTuples

in order for type to be enforced, we need a seperate type checking tool like mypy.


In [22]:
from typing import NamedTuple

class Car(NamedTuple):
    color: str
    mileage: float
    automatic: bool

car1 = Car('red', 3812.4, True)

# Type annotations are not enforced without
# a separate type checking tool like mypy:
Car('red', 'NOT_A_FLOAT', 99)

Car(color='red', mileage='NOT_A_FLOAT', automatic=99)

### `struct.Struct` serialized C structs

Structs are defined using a format strings-like mini language that allows you to define the arrangement of various C data types like char, int, and long, as well as their unsigned variants.

They’re intended primarily as a data exchange format, rather than as a way of holding data in memory that’s only used by Python code.

not recommended

In [23]:
from struct import Struct

MyStruct = Struct('i?f')
data = MyStruct.pack(23, False, 42.0)
print(data)

MyStruct.unpack(data)

b'\x17\x00\x00\x00\x00\x00\x00\x00\x00\x00(B'


(23, False, 42.0)

### `types.SimpleNamespace` Fancy Attribute Access

It provides attribute access to its namespace. This means SimpleNamespace instances expose all of their keys as class attributes. This means you can use obj.key “dotted” attribute access instead of the obj['key'] square-brackets indexing syntax that’s used by regular dicts.

It’s basically a glorified dictionary that allows attribute access and prints nicely. Attributes can be added, modified, and deleted freely.


In [26]:
from types import SimpleNamespace
car1 = SimpleNamespace(color='red', mileage=3812.4, automatic=True)

car1.mileage = 12
car1.windshield = 'broken'
del car1.automatic

car1

namespace(color='red', mileage=12, windshield='broken')

## Sets and Multisets

### `set` your go-to set

### `frozenset` Immutable Sets
can be used as dict keys (set cann't).

### `collections.Counter` Multisets

allows elements in the set to have more than one occurrence. useful if need to track how many times an element is included in the set.

note, `len` returns the number of unique elements in the multiset, use `sum` to get the total number of elements.


In [27]:
vowels = {'a', 'e', 'i', 'o', 'u'}
squares = {x * x for x in range(10)}

# empty set must be created using set(), you cannot do {}


vowels = frozenset({'a', 'e', 'i', 'o', 'u'})
d = { frozenset({1, 2, 3}): 'hello' }

In [30]:
from collections import Counter
inventory = Counter()
loot = {'sword': 1, 'bread': 3}
inventory.update(loot)
print(inventory)

more_loot = {'sword': 1, 'apple': 1}
inventory.update(more_loot)
print(inventory)

print(len(inventory))
print(sum(inventory.values()))

Counter({'bread': 3, 'sword': 1})
Counter({'bread': 3, 'sword': 2, 'apple': 1})
3
6


## Stacks (LIFO)

DFS use stacks

### `list` as Stacks

list add is ammortized O(1), but only if append to end and pop from end. From front is much slower O(n)

### `collections.deque` Fast and Robust Stacks

The deque class implements a double-ended queue that supports adding and removing elements from either end in O(1) time (nonamortized). Because deques support adding and removing elements from either end equally well, they can serve both as queues and as stacks.

access is O(n).

### `queue.LifoQueue` locking semantics for parallel computing.

This stack implementation in the Python standard library is synchronized and provides locking semantics to support multiple concurrent producers and consumers.



In [31]:
from queue import LifoQueue

s = LifoQueue()
s.put('eat')
s.put('sleep')
s.put('code')
print(s)

s.get()
s.get()
s.get_nowait()

'eat'

## Queues (FIFOs)

Unlike lists or arrays, queues typically don’t allow for random access to the objects they contain. 

BFS use queues.

### `list` very slow queues

### `collections.deque` fast and robust queues

Python’s deque objects are implemented as doubly-linked lists.

### `queue.Queue` – Locking Semantics for Parallel Computing

This queue implementation in the Python standard library is synchronized and provides locking semantics to support multiple concurrent producers and consumers.

### `multiprocessing.Queue` – Shared Job Queues

This is a shared job queue implementation that allows queued items to be processed in parallel by multiple concurrent workers. Processbased parallelization is popular in CPython due to the global interpreter lock (GIL) that prevents some forms of parallel execution on a single interpreter process.

As a specialized queue implementation meant for sharing data between processes, multiprocessing.Queue makes it easy to distribute work across multiple processes in order to work around the GIL limitations. This type of queue can store and transfer any pickle-able object across process boundaries.



In [36]:
from collections import deque

q = deque()
q.append('eat')
q.append('sleep')
q.append('code')
q.popleft()

from queue import Queue

q = Queue()
q.put('eat')
q.put('sleep')
q.put('code')
print(q)

print(q.get())

print(q.get_nowait())

from multiprocessing import Queue
q = Queue()

q.put('eat')
q.put('sleep')
q.put('code')

print(q)
print(q.get())



eat
sleep
eat


## Priority queues

A priority queue is a container data structure that manages a set of
records with totally-ordered39 keys (for example, a numeric weight
value) to provide quick access to the record with the smallest or largest
key in the set.

You can think of a priority queue as a modified queue: instead of retrieving
the next element by insertion time, it retrieves the highestpriority
element. The priority of individual elements is decided by
the ordering applied to their keys.

Scheduling algorithms often use priority queues internally. These are
specialized queues: Instead of retrieving the next element by insertion
time, a priority queue retrieves the highest-priority element. The priority
of individual elements is decided by the queue, based on the ordering
applied to their keys.

### `list` Maining a manually sorted queue

While the insertion point can be found in O(log n) time using
bisect.insort in the standard library, this is always dominated
by the slow insertion step.

Maintaining the order by appending to the list and re-sorting also
takes at least O(n log n) time.


`heapq` – List-Based Binary Heaps

This is a binary heap implementation usually backed by a plain list,
and it supports insertion and extraction of the smallest element in
O(log n) time.

This module is a good choice for implementing priority queues in
Python. Since heapq technically only provides a min-heap implementation,
extra steps must be taken to ensure sort stability and other
features typically expected from a “practical” priority queue.


In [37]:
import heapq
q = []
heapq.heappush(q, (2, 'code'))
heapq.heappush(q, (1, 'eat'))
heapq.heappush(q, (3, 'sleep'))

while q:
    next_item = heapq.heappop(q)
    print(next_item)

(1, 'eat')
(2, 'code')
(3, 'sleep')


### `queue.PriorityQueue` – Beautiful Priority Queues

This priority queue implementation uses heapq internally and shares the same time and space complexities. 

The difference is that PriorityQueue is synchronized and provides locking semantics to support multiple concurrent producers and consumers. the class based interface maybe preferred over `heapq`

In [38]:
from queue import PriorityQueue
q = PriorityQueue()
q.put((2, 'code'))
q.put((1, 'eat'))
q.put((3, 'sleep'))
while not q.empty():
    next_item = q.get()
    print(next_item)

(1, 'eat')
(2, 'code')
(3, 'sleep')


# Looping and Iteration

## Writting Pythonic Loops

## Comprehending Comprehensions

* for set:
`{ x * x for x in range(-9, 10) }`

* for dictionary
`{ x * x for x in range(-9, 10) }`

### List Slicing Tricks and the Sushi Operator (:)

`[start:stop:step]`, [::-1] means reverse.

to my surprise, [:] is shallow copy (only the structure of the elements is copied, not the elements themselves. Both copies share the same instances of elements). Use copy module for deep copy.

## Iterators

python iterator protocol is `__iter__` and `__next__`.

### Iterating Forever

1. In the __init__ method, we link each RepeaterIterator instance to the Repeater object that created it. That way we can hold onto the “source” object that’s being iterated over.
2. In RepeaterIterator.__next__, we reach back into the “source” Repeater instance and return the value associated with it.

### How do for-in loops work in Python

The for loop is a syntax sugar for the below while loop:

* It first prepared the repeater object for iteration by calling its `__iter__` method. This returned the actual iterator object.
* After that, the loop repeatedly called the iterator object’s `__next__` method to retrieve values from it.

this is like database, first we initialize the cursor, second we fetch data from it.

In [45]:
repeater = Repeater('Hello')
iterator = repeater.__iter__()
while True:
    item = iterator.__next__()
    break
    print(item)

# this also is equivlanet to
repeater = Repeater('Hello')
iterator = iter(repeater)
next(iterator)
next(iterator)

'Hello'

* as there is never more than 1 element in flight, the iterator is highly memory-efficient. so it can handle infinite sequences.
* iterator allows you iterate every element while being completely isolated from the container's internal structure. (think about c++ iterator)

### A simple iterator class

We needed `RepeaterIterator` to host the `__next__` method for fetching new values from the iterator. But it doesn’t really matter where `__next__` is defined. In the iterator protocol, all that matters is that `__iter__` returns any object with a __next__ method on it.



In [47]:
class Repeater:
    def __init__(self, value):
        self.value = value
    def __iter__(self):
        return self
    def __next__(self):
        return self.value

repeater = Repeater('Hello')
for item in repeater:
    print(item)
    break

Hello


Iterators use exceptions `StopIteration` to structure control flow. Python iterators normally can’t be “reset”—once they’re exhausted they’re supposed to raise StopIteration every time next() is called on them. To iterate anew you’ll need to request a fresh iterator object with the iter() function.

In [49]:
class BoundedRepeater:
    def __init__(self, value, max_repeats):
        self.value = value
        self.max_repeats = max_repeats
        self.count = 0
    def __iter__(self):
        return self
    def __next__(self):
        if self.count >= self.max_repeats:
            raise StopIteration
        self.count += 1
        return self.value

## Generators are simplified iterators

### infinit iterators

In [50]:
def repeater(value):
    while True:
        yield value

## calling the above gives a generator
repeater('hey')

<generator object repeater at 0x7f2505fd5250>

the generator only executes when `next()` when called. It is a syntax sugar for iterator protocal.

In [51]:
next(repeater('hey'))

'hey'

### generators that stop generating

Generators stop generating values as soon as control flow passes last yield statement.

In [54]:
def repeat_three_times(value):
    yield value
    yield value
    yield value

for x in repeat_three_times('Hey there'):
    print(x)

def bounded_repeater(value, max_repeats):
    count = 0
    while True:
        if count >= max_repeats:
            return
        count += 1
        yield value

def bounded_repeater(value, max_repeats):
    for i in range(max_repeats):
        yield value

Hey there
Hey there
Hey there


## Generator Expressions

Generator expressions are somewhat similar to list comprehensions. It is a more effective shortcut for writing iterators. Unlike list comprehensions, however, generator expressions don’t construct list objects.

`genexpr = (expression for item in collection if condition)`

In [57]:
iterator = ('Hello' for i in range(3))
print(iterator)
print(next(iterator))

<generator object <genexpr> at 0x7f2506a66b50>
Hello


### In-line Generator Expressions

() can be ignored inline:
`sum(x * 2 for x in range(10))`

In [ ]:
(expr for x in xs if cond1
    for y in ys if cond2
#    ...
    for z in zs if condN)

#this is equivalent to
for x in xs:
    if cond1:
        for y in ys:
            if cond2:
            ...
                for z in zs:
                    if condN:
                        yield expr


## Iterator Chains

By chaining together multiple iterators you can write highly efficient data processing “pipelines.” The only downside to using generator expressions is that they can’t be configured with function arguments, and you can’t reuse the same generator expression multiple times in the same processing pipeline.

In [58]:
def integers():
    for i in range(1, 9):
        yield i

def squared(seq):
    for i in seq:
        yield i * i

def negated(seq):
    for i in seq:
        yield -i

chain = negated(squared(integers()))
list(chain)

[-1, -4, -9, -16, -25, -36, -49, -64]

In [59]:
integers = range(8)
squared = (i * i for i in integers)
negated = (-i for i in squared)
print(negated)

<generator object <genexpr> at 0x7f25066d1750>
